# HDA - Project 3

In [6]:
import utils
import deeplearning
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

from keras import regularizers
from keras.activations import relu
from keras.layers import Conv2D, BatchNormalization, Dropout, LeakyReLU, Flatten, Activation, Dense, MaxPooling2D, LSTM, Reshape
from keras.models import Model, Sequential
from keras.optimizers import Adam

The following cell contains the hyper-parameters that can be tuned for code execution:
- subject: select the subject on which to test the model, between [1,4];
- folder: directory name where '.mat' files are stored;
- label_col: column of features to be selected to perform activity detection, between [0,6]:

|  Label |  Feature |
|:-:     |:-:|
|  0     | Locomotion (TASK A)  |
|  1     | High Level Activity |
|  2     | Low Level Left Arm  |
|  3     | Low Level Left Arm Object  |
|  4     | Low Level Right Arm  |
|  5     | Low Level Right Arm Object  |
|  6     | Medium Level Both Arms (TASK B2) |

- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

The size of the temporal window seems to be fundamental in order to get a more specific and powerful model; of course the choice of the step lenght between consequent windows has to be consistent and to make sense. Thinking about a real-time situation, as long as we collect data we can use a sliding window of real-time samples; in this way, it is reasonable to use also a small value for the stride. Another important reason behind the choice of the value of the 

In [7]:
subject = 1
folder = "./data/reduced/"
#folder = "/floyd/input/hdadataset/full/" # To be used with FloydHub
label = 0     # default for task A
window_size = 64
stride = 3
null_class = True

In [8]:
[x_train, y_train, x_test, y_test, n_classes] = utils.preprocessing(subject,
                                                         folder,
                                                         label,
                                                         window_size,
                                                         stride,
                                                         null_class)


Session shapes:
ADL1:   (45810, 58)
ADL2:   (28996, 58)
ADL3:   (30167, 58)
ADL4:   (30228, 58)
ADL5:   (27308, 58)
Drill:  (52152, 58)
Training samples:  157125 
Test samples:       57536 
Features:             58
TRAINING SET:
Features have shape:  (52354, 64, 58) 
Labels have shape:    (52354, 5) 
Fraction of labels:   [0.10988654 0.41987241 0.27476411 0.17119991 0.02427704]
TEST SET:
Features have shape:  (19157, 64, 58) 
Labels have shape:    (19157, 5) 
Fraction of labels:   [0.17742862 0.34337318 0.20290233 0.23771989 0.03857598]


# Hybrid Neural Network
In the following section we have implemented the neural network proposed in [1], with one block consisting of (Batch Normalization + Conv2D + MaxPool2D) followed by two layers of **LSTM**. 

Particular emphasis should be given to how we pass the output of the convolutional block to the recurrent layers; in fact, we need to appropriately reshape the output in order to make it consistent. As we know, the _convolutional power_  consists on being able to extract significant features from our input data; with the reshape procedure we merge these features in order to give them a sense of temporal dependency.

Practical example using the following problem: in our case, our samples consist of 64x110 images and we implemented 50 distinct filters in the convolutional block. So, after the conv block, our images are represented by a 27x110x50 tensor. In order to shift this in the _temporal domain_ we consider the 27 lines as temporal dependant and we represent each line as a feature vector of size 110x50. After this procedure we can feed the recurrent neural network appropriately. 

In [12]:
def HybridFeatsCorr(input_shape, classes, withSoftmax = True):
    
    model = Sequential()
  
    # Layer 0
    model.add(BatchNormalization(input_shape = input_shape))

    # Layer 1
    model.add(Conv2D(filters = 50,
                    kernel_size = (11,3),
                    strides=(1,1),
                    activation='relu'))
    
    # Layer 2
    model.add(MaxPooling2D(pool_size=(2,1)))
    
    # Layer 3
    # This layer dimension are automatically scanned in order to avoid updating by hand each time
    model.add(Reshape((model.layers[2].output_shape[1],model.layers[2].output_shape[2] * model.layers[2].output_shape[3])))  

    # Layer 4
    model.add(LSTM(300,
                  return_sequences=True))
    
    # Layer 5 
    model.add(LSTM(300))
   
    # Layer 6
    model.add(Dense(512,activation = 'relu'))
    
    if (withSoftmax):
        # Layer 7
        model.add(Dense(classes, activation = 'softmax'))
    
    return model

In [13]:
n_features = x_train.shape[2] #number of features taken into consideration for the solution of the problem

model_hyb = HybridFeatsCorr((window_size,n_features,1), n_classes)
model_hyb.summary() # model visualization

model_hyb.compile(optimizer = Adam(lr=0.01), 
                   loss = "categorical_crossentropy", 
                   metrics = ["accuracy"])

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 64, 58, 1)         4         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 54, 56, 50)        1700      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 27, 56, 50)        0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 27, 2800)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 27, 300)           3721200   
_________________________________________________________________
lstm_4 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               154112    
__________

Just an implementation note: if we want to use the 2D convolution, we have to add to our input the _depth_ information. Only out dataset representation changes, not its size.

In [ ]:
model_hyb.fit(x = input_train, 
               y = y_train, 
               epochs = 25, 
               batch_size = 300,
               verbose = 1,
               validation_data=(input_test, y_test))

In [7]:
[trainingFeatures, testingFeatures] = deeplearning.extractFeatures(model,
                                                                   input_train,
                                                                   input_test,
                                                                   model_hyb.layers[-1].output_shape[1],
                                                                   batchSize = 300)

np.save('./features_training.npy',trainingFeatures)
np.save('./labels_training.npy',y_train)
np.save('./features_testing.npy',testingFeatures)
np.save('./labels_testing.npy',y_test)

Computing DNN features on the training set...


TRAINING: 100%|#####################################################| 88/88 [06:30<00:00,  4.44s/it]


Computing DNN features on the testing set...


TESTING: 100%|######################################################| 32/32 [02:08<00:00,  4.02s/it]


In [ ]:
# Reverse the one-hot encoder procedure in order to obtain the output labels
output_train = np.argmax(y_train, axis=1)
output_test = np.argmax(y_test, axis=1)
prediction_encoded = model.predict(input_test)
prediction = np.argmax(prediction_encoded, axis=1)

C = [2**(-6)]
prediction_svm = deeplearning.SVMLayer(C,
                                        output_train,
                                        trainingFeatures,
                                        testingFeatures) 

In [ ]:
print("\nBEFORE SVM:")
print("Accuracy: ", accuracy_score(output_test, prediction))
print("F1-measure: ", utils.f1_score(output_test, prediction, average='weighted'))

print("\nAFTER SVM:")
print("Accuracy: ", accuracy_score(output_test, prediction_svm))
print("F1-measure: ", utils.f1_score(output_test, prediction_svm, average='weighted'))